# Objetivo del Notebook:

En este notebook convertimos muchos archivo de texto que contienen las intervenciones de los diputados en el Congreso en un dataframe que clasifica cada intervención según su fecha, nombre de diputado y agrupación política.

## Datos de los diputados

Disponemos de un archivo JSON con todos los datos de los diputados

In [1]:
route = r""

In [2]:
import json

f = open(route, encoding="utf8")

file_data = json.load(f)
diputados = file_data["diputados"]
cargos = file_data["cargos"]
print(diputados)
print(cargos)

[{'apellidos': 'Abades Martínez', 'formacion': 'PP', 'apellidosNombre': 'Abades Martínez, Cristina', 'fchBaja': '', 'genero': 2, 'fchAlta': '08/08/2023', 'idLegislatura': 15, 'grupo': 'Grupo Parlamentario Popular en el Congreso', 'idCircunscripcion': 27, 'nombreCircunscripcion': 'Lugo', 'nombre': 'Cristina', 'codParlamentario': 160}, {'apellidos': 'Abad Pérez', 'formacion': 'PP', 'apellidosNombre': 'Abad Pérez, Juan Antonio', 'fchBaja': '27/10/2015', 'genero': 1, 'fchAlta': '29/11/2011', 'idLegislatura': 10, 'grupo': 'Grupo Parlamentario Popular en el Congreso', 'idCircunscripcion': 26, 'nombreCircunscripcion': 'La Rioja', 'nombre': 'Juan Antonio', 'codParlamentario': 87}, {'apellidos': 'Abad Pinillos', 'formacion': 'PSOE', 'apellidosNombre': 'Abad Pinillos, María Isabel', 'fchBaja': '06/06/1995', 'genero': 2, 'fchAlta': '10/05/1994', 'idLegislatura': 5, 'grupo': 'Grupo Socialista del Congreso', 'idCircunscripcion': 9, 'nombreCircunscripcion': 'Burgos', 'nombre': 'María Isabel', 'codPa

## Identificando por nombre

Implementamos un algoritmo que mide el parecido entre 2 strings para solventar errores tipográficos

In [3]:
from difflib import SequenceMatcher

SequenceMatcher(None, "fuga ibar3ne", "fraga iribarne").ratio()

0.7692307692307693

In [4]:
# Esta función encuentra el mejor match de un tag con un diccionario de nombres

def best_match(tag, diputados):

    l = -1
    r = -1

    for i in range(len(tag)):
        if tag[i] == "(":
            l = i
    for i in range(len(tag)):
        if tag[i] == ")":
            r = i

    match_name = None
    match_gender = None
    match_grupo = None
    maximum = 0
    final_diputado = {}
    
    for diputado in diputados:
        flag = 0
        tmp = SequenceMatcher(None, diputado["apellidos"].lower(), tag[9:l].lower()).ratio()
        if (tmp > maximum):
            maximum = tmp
            flag = 1
        tmp = SequenceMatcher(None, diputado["apellidos"].lower(), tag[l+1:r].lower()).ratio()
        if (tmp > maximum):
            maximum = tmp
            flag = 1
        tmp = SequenceMatcher(None, diputado["apellidos"].lower(), tag[9:].lower()).ratio()
        if (tmp > maximum):
            maximum = tmp
            flag = 1
        if (flag):
            match_name = diputado["apellidos"]
            match_grupo = diputado["formacion"]
            match_gender = diputado["genero"]
            final_diputado = diputado

    return {"name": match_name, "group": match_grupo, "gender": match_gender, "similarity": maximum, "diputado": final_diputado}

In [5]:
best_match("El señor FUGA IBARNE:", diputados)["diputado"]

{'apellidos': 'Fraga Iribarne',
 'formacion': 'CP',
 'apellidosNombre': 'Fraga Iribarne, Manuel',
 'fchBaja': '03/07/1987',
 'genero': 1,
 'fchAlta': '09/07/1986',
 'idLegislatura': 3,
 'grupo': 'Grupo Parlamentario Coalición Popular',
 'idCircunscripcion': 28,
 'nombreCircunscripcion': 'Madrid',
 'nombre': 'Manuel',
 'codParlamentario': 206}

## Extrayendo los txt

In [6]:
route = r""

In [7]:
import os
import pandas as pd
# Ejemplo de iteración sobre un diario

with open(os.path.join(route, "PL_5_13.txt"), "r", encoding="utf8") as f:
    lines = f.readlines()
    tags = [lines[i] for i in range(len(lines)) if i % 2 == 0]
    matches = []

    cache = [] # Cache de los diputados que intervienen en este diario (para reducir el número de consultas de parecido)

    for tag in tags:
        quick_match = best_match(tag, cache)
        cargo_match = best_match(tag, cargos)

        # Comprobamos si la intervención viene de un puesto de organización del pleno (presidente, secretario, ...)
        if cargo_match["similarity"] > 0.7:
            matches.append(cargo_match["name"])
            continue
        
        # Comprobamos la similitud de los diputados que ya han intervenido en este diario
        if quick_match["similarity"] > 0.7:
            matches.append(quick_match["name"])
            continue
        
        match = best_match(tag, diputados)
        matches.append(match["name"])
        cache.append(match["diputado"])

    df = pd.DataFrame(tags, matches)
df

,0
Presidente,El señor VICEPRESIDENTE (Beviá Pastor)\n
Robles Fraga,El señor ROBLES FRAGA\n
Presidente,El señor VICEPRESIDENTE (Beviá Pastor)\n
Solana Madariaga,El señor MINISTRO DE ASUNTOS EXTERIORES (Solan...
Presidente,El señor VICEPRESIDENTE (Beviá Pastor)\n
...,...
Molins i Amat,El señor MOLINS 1 AMAT\n
Presidente,El señor PRESIDENTE. \n
Rajoy Brey,El señor RAJOY BREY\n
Presidente,El señor PRESIDENTE\n


## Iteración sobre el set

In [8]:
import os
from difflib import SequenceMatcher
import numpy as np
import pandas as pd

# SequenceMatcher(None, "NU-", "NúM").ratio()

dir = route
legislatura = 1

# Extraemos las diferentes columnas
# Eliminaremos algunas de ellas posteriormente porque no son relevantes para este trabajo

legislature = []
number = []
text = []
name = []
group = []
gender = []

while legislatura <= 15:
    num = 1
    while num <= 400:
        try:
            # Accedemos a cada diario según su legislatura y número de diario
            with open(os.path.join(dir,"PL_"+str(legislatura)+"_"+str(num)+".txt"), "r", encoding="utf8") as f: 
                lines = f.readlines()
                cache = []
                for i in range(len(lines)):
                    if i % 2 != 0:
                        continue

                    tag = lines[i]
                    text.append(lines[i+1])
                    legislature.append(legislatura)
                    number.append(num)

                    cargo_match = best_match(tag, cargos)
                    if cargo_match["similarity"] > 0.7:
                        name.append(quick_match["name"])
                        group.append(quick_match["group"])
                        gender.append(quick_match["gender"])
                        continue  

                    quick_match = best_match(tag, cache)
                    if quick_match["similarity"] > 0.7:
                        name.append(quick_match["name"])
                        group.append(quick_match["group"])
                        gender.append(quick_match["gender"])
                        continue

                    match = best_match(tag, diputados)
                    if match["similarity"] > 0.7:
                        name.append(match["name"])
                        group.append(match["group"])
                        gender.append(match["gender"])
                        cache.append(match["diputado"])
                    else:
                        name.append(None)
                        group.append(None)
                        gender.append(None)
        except:
            # Faltan uno o dos diarios por legislatura, si no se encuentran en los archivos se continua.
            num+=1
            continue

        if num % 50 == 0:
            print(num, end=" ")
        num+=1

    print("\n", "LEGISLATURA ",  legislatura, "ANALIZADA")
    legislatura+=1
    
df = pd.DataFrame({
    "text": pd.array(text, dtype="string"),
    "number": number,
    "legislature": legislature,
    "group": pd.Categorical(group),
    "name": pd.array(name, dtype="string"),
    "gender": gender
})

50 100 150 200 250 
 LEGISLATURA  1 ANALIZADA
50 100 200 250 
 LEGISLATURA  2 ANALIZADA
50 100 150 200 
 LEGISLATURA  3 ANALIZADA
50 100 150 200 250 
 LEGISLATURA  4 ANALIZADA
50 100 150 
 LEGISLATURA  5 ANALIZADA
50 100 150 200 250 
 LEGISLATURA  6 ANALIZADA
50 100 150 200 250 300 
 LEGISLATURA  7 ANALIZADA
50 100 150 200 250 300 
 LEGISLATURA  8 ANALIZADA
50 100 150 
 LEGISLATURA  9 ANALIZADA
50 100 150 200 250 300 
 LEGISLATURA  10 ANALIZADA

 LEGISLATURA  11 ANALIZADA
50 100 150 
 LEGISLATURA  12 ANALIZADA

 LEGISLATURA  13 ANALIZADA
50 100 150 200 250 
 LEGISLATURA  14 ANALIZADA

 LEGISLATURA  15 ANALIZADA


In [10]:
df

,text,number,legislature,group,name,gender
0,De acuerdo con los datos que obran en la Secre...,1,1,NaN,<NA>,NaN
1,"Señoras y señores Diputados, se abre la sesión...",1,1,NaN,<NA>,NaN
2,sEENERAL DEL CONGRESO DE LOS DIPUTADOS (Rubio ...,1,1,NaN,<NA>,NaN
3,El artículo que hace referencia a este acto es...,1,1,NaN,<NA>,NaN
4,Según la relación que consta en la Secretaría ...,1,1,NaN,<NA>,NaN
...,...,...,...,...,...,...
432296,La estamos estudiando. Muchas gracias.,43,15,VOX,Hoces Íñiguez,1.0
432297,De acuerdo. Votamos en un minuto. (Pausa). TOM...,43,15,VOX,Hoces Íñiguez,1.0
432298,"Señoras y señores diputados, vamos a proceder ...",43,15,VOX,Hoces Íñiguez,1.0
432299,"En consecuencia, se toma en consideración esta...",43,15,VOX,Hoces Íñiguez,1.0


## Fecha de las intervenciones

Tenemos un csv en el que tenemos la fecha de cada intervención

In [11]:
route = r""
dates = pd.read_csv(route)

In [12]:
dates_dict = dict()
dates_column = []

# Convertimos el df en un diccionario en el que el diario nos devuelve la fecha
for i in range(len(dates)):
    x = str(dates.loc[i, "0"])
    dates_dict[(dates.loc[i, "Unnamed: 0"], dates.loc[i, "Unnamed: 1"])] = pd.Timestamp(year=int(x[:4]),month=int(x[4:6]),day=int(x[6:]))
# Creamos una columna con las fechas
for i in range(len(df)):
    dates_column.append(dates_dict[(df.loc[i, "legislature"], df.loc[i, "number"])])
len(dates_column)

df["date"] = dates_column

## Ajuste y exportación del dataframe

Recortamos el dataframe para las necesidades del trabajo

In [13]:
## Cambiamos el género a un formato más legible

import numpy as np
gender_column = []
for i in range(len(df)):
    x = df.loc[i, "gender"]
    if x == 1:
        gender_column.append("Male")
    elif x == 2:
        gender_column.append("Female")
    else:
        gender_column.append("Unknown")
gender_column

df["gender"] = gender_column

In [14]:
import pandas as pd
df = pd.read_csv(r"")

In [15]:
## Nos quedamos con los años
def truncate(s):
    return s[0:4]
df["year"] = df["date"].apply(truncate)

In [ ]:
# df.drop(columns={"date", "number"})

In [29]:
df[(df["name"].notnull()) & (df["group"] != "NEUTRAL")]

,text,legislature,group,name,gender,monthly_intervals,year
1,"Señoras y señores Diputados, se abre la sesión...",1,CP,Fraile Poujade,Male,1979-03,1979
4,Según la relación que consta en la Secretaría ...,1,CP,Fraile Poujade,Male,1979-03,1979
5,"Siguiendo el orden del día, ruego al señor Esp...",1,PSOE,Rodríguez Valverde,Male,1979-03,1979
6,Señoras y señores Diputados. en nombre de la M...,1,PSOE,Rodríguez Valverde,Male,1979-03,1979
7,"Al reanudarse la sesión, procedería dar lectur...",1,PSOE,Rodríguez Valverde,Male,1979-03,1979
...,...,...,...,...,...,...,...
432288,"Muchas gracias, presidenta. Buenas tardes, señ...",15,PP,Gallardo Barrena,Male,2024-05,2024
432290,"Bé, molt bona tarda a totes i a tots. Ja veig ...",15,ERC,Jordà i Roura,Female,2024-05,2024
432292,"Buenas tardes, noches ya, señorías. Probableme...",15,SUMAR,Valero Morales,Male,2024-05,2024
432294,"Gracias, presidenta. Buenas tardes, señorías. ...",15,PsdeG-PSOE,Otero Rodríguez,Female,2024-05,2024


In [30]:
df.to_csv(r"", index=False)